In [1]:
import numpy as np

In [2]:
np.random.seed(42)

In [3]:
x = (np.random.uniform(0,15,size=[100,128]))
y = (np.random.uniform(0,15,size=[100,128]))

# Distancias entre vectores

* [Distancia coseno](http://reference.wolfram.com/language/ref/CosineDistance.html):
$$
CosineDistance[\vec{u},\vec{v}] = 1 - \frac{\vec{u}\cdot\vec{v}}{\lVert \vec{u} 
\lVert \lVert \vec{v} \lVert}
$$

---

* [Distancia euclidiana](http://reference.wolfram.com/language/ref/EuclideanDistance.html):
$$
EuclidianDistance[\vec{u},\vec{v}] = \sqrt{ \sum_{k=1}^n (u_i-v_i)^2}
$$

In [4]:
def get_distance(vector, matrix,distance="cos"):
    
    # Distancia coseno
    if distance == "cos":
        return [(i,1 - ((np.dot(vector,matrix[i]))/(np.linalg.norm(vector)*np.linalg.norm(matrix[i]))))
                for i in range(matrix.shape[0])]
    
    # Distancia euclidiana
    return [(i,np.linalg.norm(vector-matrix[i]))
            for i in range(matrix.shape[0])]

In [5]:
dist = get_distance(x[0],x)

In [6]:
dist.sort(key=lambda tup: tup[1])

In [7]:
dist[:10]

[(0, 0.0),
 (82, 0.20857466632475141),
 (59, 0.22249004041280496),
 (62, 0.22287385665564552),
 (45, 0.22713294845550447),
 (87, 0.22941440680510716),
 (36, 0.23318292386386985),
 (43, 0.23490387496743415),
 (51, 0.23539674058696169),
 (86, 0.23618406303260064)]